# Model testing

This notebook provides a set of tests to run against `treat_sim`.  These tests are either pass or fail and no interpretation is needed. A summary of test results is provided at the end of the notebook.

> This notebook is a work in progress.

We have broken the testing into the 

## 1. Model Code Imports

In [1]:
from treat_sim.model import (
    Scenario, 
    TreatmentCentreModel,
    single_run, 
    multiple_replications
)

## 2. Imports

In [2]:
import numpy as np
import pandas as pd
import statistics
import pytest
import ipytest
ipytest.autoconfig()

## 3. Tests

### 3.1 Model run test

Here we test that various modes of running the model work correctly.  These include

* single run mode
* repeatable results using random number sets.
* results collection period



In [3]:
def test_single_run_type():
    '''
    Test a single_run of the model.
    
    The single_run function should return a pandas.DataFrame
    containing 16 columns and a single row.
    
     0   00_arrivals                    
     1   01a_triage_wait                 
     2   01b_triage_util               
     3   02a_registration_wait         
     4   02b_registration_util        
     5   03a_examination_wait          
     6   03b_examination_util          
     7   04a_treatment_wait(non_trauma)  
     8   04b_treatment_util(non_trauma)  
     9   05_total_time(non-trauma)       
     10  06a_trauma_wait               
     11  06b_trauma_util                 
     12  07a_treatment_wait(trauma)      
     13  07b_treatment_util(trauma)      
     14  08_total_time(trauma)           
     15  09_throughput                   

    Expected result: 
    ---------------
        len(run_results) == 16 and isinstance(run_results, pd.Dataframe)

    Returns:
    -------
    bool: does the model pass the test.
    '''
    EXPECTED_LENGTH = 16

    # a default experiment
    default_experiment_params = Scenario(random_number_set=41)

    # run the model in single run model
    run_results = single_run(default_experiment_params)

    # test
    assert len(run_results.T) == EXPECTED_LENGTH and isinstance(run_results, pd.DataFrame)
    

In [4]:
@pytest.mark.parametrize('random_number_set', [
                          (0),
                          (1),
                          (2),
                          (101),
                          (42),
])
def test_random_number_set(random_number_set):
    '''
    Test the model produces repeatable results
    given the same set set of random seeds.
    
    Expected result: 
    ---------------
        difference between data frames is 0.0
    '''

    results = []

    for i in range(2):
        
        exp = Scenario()

        # run the model in single run model
        run_results = single_run(exp, random_no_set=random_number_set)
    
        results.append(run_results)
        
    # test
    assert (results[0] - results[1]).sum().sum() == 0.0

In [5]:
@pytest.mark.parametrize('rc_period', [
                          (10.0),
                          (1_000.0),
                          (25.0),
                          (500.0),
                          (143.0),
])
def test_run_length_control(rc_period):
    scenario = Scenario()
    
    # set random number set - this controls sampling for the run.
    scenario.set_random_no_set(42)

    # create an instance of the model
    model = TreatmentCentreModel(scenario)

    # run the model
    model.run(results_collection_period=rc_period)
        
    # run results
    assert model.env.now == rc_period

## Run all automated tests

In [6]:
ipytest.run("-vv", "--no-header")

======================================= test session starts ========================================
collecting ... collected 11 items

t_e51d2d1d981b4a758ee6ead7bc5df832.py::test_single_run_type PASSED                           [  9%]
t_e51d2d1d981b4a758ee6ead7bc5df832.py::test_random_number_set[0] PASSED                      [ 18%]
t_e51d2d1d981b4a758ee6ead7bc5df832.py::test_random_number_set[1] PASSED                      [ 27%]
t_e51d2d1d981b4a758ee6ead7bc5df832.py::test_random_number_set[2] PASSED                      [ 36%]
t_e51d2d1d981b4a758ee6ead7bc5df832.py::test_random_number_set[101] PASSED                    [ 45%]
t_e51d2d1d981b4a758ee6ead7bc5df832.py::test_random_number_set[42] PASSED                     [ 54%]
t_e51d2d1d981b4a758ee6ead7bc5df832.py::test_run_length_control[10.0] PASSED                  [ 63%]
t_e51d2d1d981b4a758ee6ead7bc5df832.py::test_run_length_control[1000.0] PASSED                [ 72%]
t_e51d2d1d981b4a758ee6ead7bc5df832.py::test_run_length_control[2

<ExitCode.OK: 0>